In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
links_paginacion = []

base = 'http://memoria2.cnpq.br/web/guest/chamadas-publicas?p_p_id=resultadosportlet_WAR_resultadoscnpqportlet_INSTANCE_0ZaM&filtro=encerradas&buscaChamada=&ano=&startPage='


for i in range(78):

    link_pagina = base + str(i)
    links_paginacion.append(link_pagina)


links_paginacion.append('http://memoria2.cnpq.br/web/guest/chamadas-publicas')

In [4]:
CNPq = pd.DataFrame()
links_vacios = []
for link in links_paginacion:
    parser_pag = parser(link)

    #Titulos
    titulos = parser_pag.xpath('////ol[@class="unstyled list-striped list-chamadas"]//li//div/h4//text()')
    #
    #Anos
    anos_brutos = parser_pag.xpath('//ol[@class="unstyled list-striped list-chamadas"]//li/div/div[@class="inscricao"]//li//text()')

    anos = []
    for a in anos_brutos:
        a = a.strip().split('/')[-1]
        anos.append(a)


    descripciones = []
    pdfs = []
    for i in range(1, len(titulos) + 1):

        #Descripcion
        descripciones_brutas = parser_pag.xpath('//ol[@class="unstyled list-striped list-chamadas"]//li[' + str(i) + ']/div[1]//p//text()[1]')
        descripcion = ''
        for d in descripciones_brutas:
            d = d.strip()
            descripcion = descripcion + d + ' '
        descripciones.append(descripcion.strip())

        #pdfs
        pdfs_brutos = parser_pag.xpath('//ol[@class="unstyled list-striped list-chamadas"]//li[' + str(i) + ']//div[@class="links-normas pull-left"]//a//@href')
        pdfs_proy = ''
        for p in pdfs_brutos:
            p = p.strip()
            pdfs_proy = pdfs_proy + p + ', '
        pdfs.append(pdfs_proy.strip(', '))


    CNPq_pag = pd.DataFrame()
    CNPq_pag['Titulo'] = titulos
    CNPq_pag['Año'] = anos
    CNPq_pag['Descripcion'] = descripciones
    CNPq_pag['PDFs'] = pdfs

    CNPq = pd.concat([CNPq,CNPq_pag])

    if len(titulos) == 0:
        links_vacios.append(link)

CNPq = CNPq.reset_index(drop=True)

In [5]:
for link in links_vacios:
    driver = webdriver.Firefox()
    driver.get(link)


    titulos_brutos = driver.find_elements_by_xpath('//ol[@class="unstyled list-striped list-chamadas"]//li//div/h4')

    titulos = []
    for tit in titulos_brutos:
        tit = tit.text
        tit = tit.strip()

        titulos.append(tit)

        #Anos
    anos_brutos = driver.find_elements_by_xpath('//ol[@class="unstyled list-striped list-chamadas"]//li/div/div[@class="inscricao"]//li[1]')

    anos = []
    for a in anos_brutos:
        a = a.text
        a = a.strip().split('/')[-1]
        anos.append(a)
    
    ##################
    descripciones = []
    pdfs = []
    for i in range(1, len(titulos) + 1):

        #Descripcion
        descripciones_brutas = driver.find_elements_by_xpath('//ol[@class="unstyled list-striped list-chamadas"]//li[' + str(i) + ']/div[1]//p')
        descripcion = ''
        for d in descripciones_brutas:
            d = d.text
            d = d.strip()
            descripcion = descripcion + d + ' '
        descripciones.append(descripcion.strip())

            #pdfs
        pdfs_brutos = driver.find_elements_by_xpath('//ol[@class="unstyled list-striped list-chamadas"]//li[' + str(i) + ']//div[@class="links-normas pull-left"]//a')
        pdfs_proy = ''
        for p in pdfs_brutos:
            p = p.get_attribute('href')
            p = p.strip()
            pdfs_proy = pdfs_proy + p + ', '
        pdfs.append(pdfs_proy.strip(', '))

    driver.quit()
        
    CNPq_pag = pd.DataFrame()
    CNPq_pag['Titulo'] = titulos
    CNPq_pag['Año'] = anos
    CNPq_pag['Descripcion'] = descripciones
    CNPq_pag['PDFs'] = pdfs

    CNPq = pd.concat([CNPq,CNPq_pag])
    #print(link)
    #print(len(anos))
    #print(len(titulos))
    #print(len(descripciones))
    #print(len(pdfs))


CNPq = CNPq.reset_index(drop=True)

In [8]:
CNPq.to_excel('brasil_CNPq.xlsx')